In [ ]:
import tensorflow as tf

In [ ]:
print(tf.__version__)

In [ ]:
import os
import io
from io import BytesIO
import tarfile
import tempfile
from six.moves import urllib

In [ ]:
import matplotlib
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
import cv2 as cv
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from tabulate import tabulate

In [ ]:
# Comment this out if you want to see Deprecation warnings
import warnings
warnings.simplefilter('ignore', DeprecationWarning)

In [ ]:
graph = tf.Graph()
graph_def = None
tar_file = tarfile.open('/media/massive/models/deeplab/deeplabv3_mnv2_cityscapes_train_2018_02_05.tar.gz')

In [ ]:
for tar_info in tar_file.getmembers():
    if 'frozen_inference_graph' in os.path.basename(tar_info.name):
        file_handle = tar_file.extractfile(tar_info)
        graph_def = tf.GraphDef.FromString(file_handle.read())
        break

In [ ]:
if graph_def is None:
    raise RuntimeError('Cannot find inference graph in tar archive.')

In [ ]:
with graph.as_default():
    tf.import_graph_def(graph_def, name='')

In [ ]:
sess = tf.Session(graph=graph)

In [ ]:
original_image = Image.open('/media/massive/datasets/mit-deep-learning/mit_driveseg_sample.png')

In [ ]:
width, height = original_image.size
target_size = (2049, 1025)
resized_image = original_image.convert('RGB').resize(target_size, Image.ANTIALIAS)

In [ ]:
batch_seg_map = sess.run(
    'SemanticPredictions:0',
    feed_dict={'ImageTensor:0': [np.asarray(resized_image)]})

In [ ]:
seg_map = batch_seg_map[0]
if len(seg_map.shape) == 2:
    seg_map = np.expand_dims(seg_map, -1)
seg_map = cv.resize(seg_map, (width,height), interpolation=cv.INTER_NEAREST)

In [ ]:
seg_map